In [1]:
from sloyka import VkCommentsParser, VkPostGetter

/home/gk/.cache/pypoetry/virtualenvs/factfinder-6OvYRBkP-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /home/gk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
owner_id = "-60463565" # группа "девяткино онлайн"
token = ''

In [3]:
post_num_limit = 10
step = 10
#Вызов метода
post_ids = VkPostGetter.get_group_post_ids(owner_id, token, post_num_limit, step)

print(len(post_ids))

10


In [4]:
comments_num = 30

# Вызов метода
result = VkCommentsParser.get_Comments(post_ids, owner_id, token, comments_num=comments_num)

 30%|███       | 3/10 [00:09<00:22,  3.24s/it]


In [5]:
result.sample(3)

,name,last_name,date,likes,text,post_id
2672680,Elena,Konik,2024-02-13,0,Кидайте свои фотки!,2672649
2672684,Nstasy,Jule,2024-02-13,1,Но это не Девяткино))) судя по дому;))🙃,2672649
2672603,Alexander,Komogortsev,2024-02-13,6,Значит она общается с другими парнями. Поверь....,2672582
